In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Prepare the data
pursuit = pd.concat(map(pd.read_csv, ["D:\\ELSEC\\SortiModel\\data\\1.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\2.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\3.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\4.csv"]), ignore_index=True)

# Data preprocessing steps
# Ensure 'Target' column exists and is the last column in the dataframe
pursuit['Target'] = pursuit.apply(lambda row: 0 if row['WeaponSystem'] == 'T5' else 1, axis=1)
pursuit = pursuit.drop(['Emitter Name'], axis=1)  # Drop 'Emitter Name' column
pursuit = pd.get_dummies(pursuit, drop_first=True)  # One-hot encode categorical variables
X = pursuit.drop('Target', axis=1)
y = pursuit['Target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the data to numpy arrays
X_train_scaled = np.array(X_train_scaled)
X_test_scaled = np.array(X_test_scaled)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

# Load the data
pursuit = pd.concat(map(pd.read_csv, ["D:\\ELSEC\\SortiModel\\data\\1.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\2.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\3.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\4.csv"]), ignore_index=True)

# Data preprocessing steps
# Ensure 'Target' column exists and is the last column in the dataframe
pursuit['Target'] = pursuit.apply(lambda row: 0 if row['WeaponSystem'] == 'T5' else 1, axis=1)
pursuit = pursuit.drop(['Emitter Name'], axis=1)  # Drop 'Emitter Name' column
pursuit = pd.get_dummies(pursuit, drop_first=True)  # One-hot encode categorical variables
X = pursuit.drop('Target', axis=1)
y = pursuit['Target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the data to numpy arrays
X_train_scaled = np.array(X_train_scaled)
X_test_scaled = np.array(X_test_scaled)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Define the TabNet model
tabnet_model = TabNetClassifier(
    n_d=64, n_a=64,
    n_steps=5,
    gamma=1.5,
    lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type='sparsemax',  # This is the default
    scheduler_params={"step_size": 10, "gamma": 0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR
)

# Train the TabNet model
tabnet_model.fit(
    X_train=X_train_scaled, y_train=y_train,
    eval_set=[(X_test_scaled, y_test)],
    eval_name=['test'],
    eval_metric=['accuracy'],
    max_epochs=5, patience=10,
    batch_size=256, virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

# Evaluate the model
accuracy = tabnet_model.predict(X_test_scaled)
accuracy_score = (accuracy == y_test).mean()
print(f"Accuracy of the model: {accuracy_score}")

D:\ELSEC\SortiModel\.venv\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.51925 | test_accuracy: 0.98706 |  0:00:15s
epoch 1  | loss: 0.06033 | test_accuracy: 1.0     |  0:00:28s
epoch 2  | loss: 0.00414 | test_accuracy: 1.0     |  0:00:39s
epoch 3  | loss: 0.00738 | test_accuracy: 0.99924 |  0:00:46s
epoch 4  | loss: 0.02366 | test_accuracy: 1.0     |  0:00:53s
Stop training because you reached max_epochs = 5 with best_epoch = 1 and best_test_accuracy = 1.0


D:\ELSEC\SortiModel\.venv\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Accuracy of the model: 1.0


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Load the data
pursuit = pd.concat(map(pd.read_csv, ["D:\\ELSEC\\SortiModel\\data\\1.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\2.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\3.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\4.csv"]), ignore_index=True)

# Data preprocessing steps
# Ensure 'Target' column exists and is the last column in the dataframe
pursuit['Target'] = pursuit.apply(lambda row: 0 if row['WeaponSystem'] == 'T5' else 1, axis=1)
pursuit = pursuit.drop(['Emitter Name'], axis=1)  # Drop 'Emitter Name' column
pursuit = pd.get_dummies(pursuit, drop_first=True)  # One-hot encode categorical variables
X = pursuit.drop('Target', axis=1)
y = pursuit['Target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the data to numpy arrays
X_train_scaled = np.array(X_train_scaled)
X_test_scaled = np.array(X_test_scaled)
y_train = np.array(y_train)
y_test = np.array(y_test)


In [4]:
pursuit = pd.concat(map(pd.read_csv, ["D:\\ELSEC\\SortiModel\\data\\1.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\2.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\3.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\4.csv"]), ignore_index=True)
clutter = pd.concat(map(pd.read_csv, ["D:\\ELSEC\\SortiModel\\data\\1.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\2.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\3.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\4.csv"]), ignore_index=True)
print(f'Size of pursuit signals: {pursuit.shape}')
print(f'Size of clutter signals: {clutter.shape}')
pursuit = pursuit.drop(pursuit[pursuit.WeaponSystem == 'T5'].index)
clutter = clutter.drop(clutter[clutter.WeaponSystem != 'T5'].index)

pursuit.loc[pursuit.WeaponSystem == 'T5', 'Target'] = 0
pursuit.loc[pursuit.WeaponSystem != 'T5', 'Target'] = 1
pursuit.loc[pursuit['Target'] > 1.0, 'Target'] = 1
pursuit['Target'] = pursuit['Target'].fillna(0)

clutter.loc[clutter.WeaponSystem == 'T5', 'Target'] = 0
clutter.loc[clutter.WeaponSystem != 'T5', 'Target'] = 1
clutter.loc[clutter['Target'] > 1.0, 'Target'] = 1
clutter['Target'] = clutter['Target'].fillna(0)
pursuit['DF Type'] = pursuit['DF Type'].replace('BLI1', 1)
pursuit['DF Type'] = pursuit['DF Type'].replace('ADF', 2)
pursuit['DF Type'] = pursuit['DF Type'].replace('BLI2', 3)
pursuit['DF Type'] = pursuit['DF Type'].replace('ADF+BLI2', 4)
pursuit['DF Type'] = pursuit['DF Type'].replace('ADF+BLI1', 5)
pursuit['DF Type'].astype(int)

pursuit['Frequency Agility'] = pursuit['Frequency Agility'].replace('Fixed Frequency', 1)
pursuit['Frequency Agility'] = pursuit['Frequency Agility'].replace('LFM', 2)
pursuit['Frequency Agility'] = pursuit['Frequency Agility'].replace('LFM (Chirp-Up)', 3)
pursuit['Frequency Agility'] = pursuit['Frequency Agility'].replace('Frequency Agile', 4)
pursuit['Frequency Agility'] = pursuit['Frequency Agility'].replace("LFM (Chirp-Up) / Frequency Agile", 5)
pursuit['Frequency Agility'] = pursuit['Frequency Agility'].replace('LFM / Frequency Agile', 6)
pursuit['Frequency Agility'].astype(int)

pursuit['Pulse Agility'] = pursuit['Pulse Agility'].replace('Fixed PRI', 1)
pursuit['Pulse Agility'] = pursuit['Pulse Agility'].replace('Fixed Frequency', 2)
pursuit['Pulse Agility'] = pursuit['Pulse Agility'].replace('Stagger PRI', 3)
pursuit['Pulse Agility'] = pursuit['Pulse Agility'].replace('Dwell&Switch PRI', 4)
pursuit['Pulse Agility'] = pursuit['Pulse Agility'].replace('Frequency Agile', 5)
pursuit['Pulse Agility'] = pursuit['Pulse Agility'].replace('LFM', 5)
pursuit['Pulse Agility'].astype(int)

pursuit['TOLA'] = pursuit['TOLA'].str.replace(' ', '').str.split(':').apply(
    lambda x: int(x[0]) * 3600 + int(x[1]) * 60 + int(x[2]))
pursuit['TOLA'] = pursuit['TOLA'].astype(int)
pursuit['TOFA'] = pursuit['TOFA'].str.replace(' ', '').str.split(':').apply(
    lambda x: int(x[0]) * 3600 + int(x[1]) * 60 + int(x[2]))
pursuit['TOFA'] = pursuit['TOFA'].astype(int)

pursuit['TD'] = (pursuit['TOLA'] - pursuit['TOFA'])
pursuit['TD'].info()

pursuit_t1 = pursuit.drop(pursuit[pursuit.WeaponSystem != 'T1'].index)
pursuit_t2 = pursuit.drop(pursuit[pursuit.WeaponSystem != 'T2'].index)
pursuit_t3 = pursuit.drop(pursuit[pursuit.WeaponSystem != 'T3'].index)
pursuit_t4 = pursuit.drop(pursuit[pursuit.WeaponSystem != 'T4'].index)

pursuit = pursuit.select_dtypes(exclude='object')
pursuit = pursuit.drop(['Emitter Name'], axis=1)
print('Pursuit Dataset Revision done !')

pursuit_t1 = pursuit_t1.select_dtypes(exclude='object')
pursuit_t1 = pursuit_t1.drop(['Emitter Name'], axis=1)
pursuit_t2 = pursuit_t2.select_dtypes(exclude='object')
pursuit_t2 = pursuit_t2.drop(['Emitter Name'], axis=1)
pursuit_t3 = pursuit_t3.select_dtypes(exclude='object')
pursuit_t3 = pursuit_t3.drop(['Emitter Name'], axis=1)
pursuit_t4 = pursuit_t4.select_dtypes(exclude='object')
pursuit_t4 = pursuit_t4.drop(['Emitter Name'], axis=1)

clutter['DF Type'] = clutter['DF Type'].replace('BLI1', 1)
clutter['DF Type'] = clutter['DF Type'].replace('ADF', 2)
clutter['DF Type'] = clutter['DF Type'].replace('BLI2', 3)
clutter['DF Type'] = clutter['DF Type'].replace('ADF+BLI2', 4)
clutter['DF Type'] = clutter['DF Type'].replace('ADF+BLI1', 5)
clutter['DF Type'].astype(int)

clutter['Frequency Agility'] = clutter['Frequency Agility'].replace('Fixed Frequency', 1)
clutter['Frequency Agility'] = clutter['Frequency Agility'].replace('LFM', 2)
clutter['Frequency Agility'] = clutter['Frequency Agility'].replace('LFM (Chirp-Up)', 3)
clutter['Frequency Agility'] = clutter['Frequency Agility'].replace('Frequency Agile', 4)
clutter['Frequency Agility'] = clutter['Frequency Agility'].replace("LFM (Chirp-Up) / Frequency Agile", 5)
clutter['Frequency Agility'] = clutter['Frequency Agility'].replace('LFM / Frequency Agile', 6)
clutter['Frequency Agility'].astype(int)

clutter['Pulse Agility'] = clutter['Pulse Agility'].replace('Fixed PRI', 1)
clutter['Pulse Agility'] = clutter['Pulse Agility'].replace('Fixed Frequency', 2)
clutter['Pulse Agility'] = clutter['Pulse Agility'].replace('Stagger PRI', 3)
clutter['Pulse Agility'] = clutter['Pulse Agility'].replace('Dwell&Switch PRI', 4)
clutter['Pulse Agility'] = clutter['Pulse Agility'].replace('Frequency Agile', 5)
clutter['Pulse Agility'] = clutter['Pulse Agility'].replace('LFM', 5)
clutter['Pulse Agility'].astype(int)

clutter['TOLA'] = clutter['TOLA'].str.replace(' ', '').str.split(':').apply(
    lambda x: int(x[0]) * 3600 + int(x[1]) * 60 + int(x[2]))
clutter['TOLA'] = clutter['TOLA'].astype(int)
clutter['TOFA'] = clutter['TOFA'].str.replace(' ', '').str.split(':').apply(
    lambda x: int(x[0]) * 3600 + int(x[1]) * 60 + int(x[2]))
clutter['TOFA'] = clutter['TOFA'].astype(int)
clutter['TD'] = (clutter['TOLA'] - clutter['TOFA'])
clutter['TD'].info()

clutter = clutter.select_dtypes(exclude='object')
clutter = clutter.drop(['Emitter Name'], axis=1)
print('Clutter Dataset Revision done !')
print('Dataset processing done !')
pursuit_test = pursuit.drop('Target', axis=1)
pursuit_res = pursuit['Target']
pursuit_t1_test = pursuit_t1.drop('Target', axis=1)
pursuit_t1_res = pursuit_t1['Target']
pursuit_t2_test = pursuit_t2.drop('Target', axis=1)
pursuit_t2_res = pursuit_t2['Target']
pursuit_t3_test = pursuit_t3.drop('Target', axis=1)
pursuit_t3_res = pursuit_t3['Target']
pursuit_t4_test = pursuit_t4.drop('Target', axis=1)
pursuit_t4_res = pursuit_t4['Target']
print(f'Size of Pursuit T1:{len(pursuit_t1)}')
print(f'Size of Pursuit T2:{len(pursuit_t2)}')
print(f'Size of Pursuit T3:{len(pursuit_t3)}')
print(f'Size of Pursuit T4:{len(pursuit_t4)}')
train_size = int(len(pursuit_t1) * 0.8)
X_train = pursuit_t1_test[:train_size]
y_train = pursuit_t1_res[:train_size]

X_test = clutter.drop('Target', axis=1)
y_test = clutter['Target']

y_train_np = np.array(y_train)
num_ones_train = (y_train_np == 1).sum()
print(f'Total Number of Ones in Target Column of the training dataset: {num_ones_train}')
print(f'Total Number of Zeros in Target Column of the training dataset: {len(y_train) - num_ones_train}')
y_test_np = np.array(y_test)
num_ones_test = (y_test_np == 1).sum()
print(f'Total Number of Ones in Target Column of the test dataset: {num_ones_test}')
print(f'Total Number of Zeros in Target Column of the test dataset: {len(y_test) - num_ones_test}')
num_ones_test + num_ones_train, (len(y_test) - num_ones_test) + (len(y_train) - num_ones_train)

Size of pursuit signals: (19696, 97)
Size of clutter signals: (19696, 97)
<class 'pandas.core.series.Series'>
Index: 7856 entries, 1 to 19692
Series name: TD
Non-Null Count  Dtype
--------------  -----
7856 non-null   int32
dtypes: int32(1)
memory usage: 92.1 KB
Pursuit Dataset Revision done !
<class 'pandas.core.series.Series'>
Index: 11840 entries, 0 to 19695
Series name: TD
Non-Null Count  Dtype
--------------  -----
11840 non-null  int32
dtypes: int32(1)
memory usage: 138.8 KB
Clutter Dataset Revision done !
Dataset processing done !
Size of Pursuit T1:1357
Size of Pursuit T2:1299
Size of Pursuit T3:4876
Size of Pursuit T4:324
Total Number of Ones in Target Column of the training dataset: 1085
Total Number of Zeros in Target Column of the training dataset: 0
Total Number of Ones in Target Column of the test dataset: 0
Total Number of Zeros in Target Column of the test dataset: 11840


C:\Users\RK Niranjan\AppData\Local\Temp\ipykernel_1904\3503905900.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pursuit['DF Type'] = pursuit['DF Type'].replace('ADF+BLI2', 4)
C:\Users\RK Niranjan\AppData\Local\Temp\ipykernel_1904\3503905900.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pursuit['Frequency Agility'] = pursuit['Frequency Agility'].replace("LFM (Chirp-Up) / Frequency Agile", 5)
C:\Users\RK Niranjan\AppData\Local\Temp\ipykernel_1904\3503905900.py:42: FutureWarning: Downcasting behavior in `replace` is dep

(1085, 11840)

In [5]:
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the data to numpy arrays
X_train_scaled = np.array(X_train_scaled)
X_test_scaled = np.array(X_test_scaled)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Apply SMOTE to balance the classes in the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the XGBoost model
xgb_model = xgb.XGBClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False
)

# Train the model
xgb_model.fit(
    X_train_scaled,
    y_train,
    eval_set=[(X_test_scaled, y_test)],
    early_stopping_rounds=10,
    verbose=True
)

# Predict on the test set
y_pred = xgb_model.predict(X_test_scaled)
y_pred_proba = xgb_model.predict_proba(X_test_scaled)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')


D:\ELSEC\SortiModel\.venv\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.64447
[1]	validation_0-logloss:0.60037
[2]	validation_0-logloss:0.56028
[3]	validation_0-logloss:0.52371
[4]	validation_0-logloss:0.49021
[5]	validation_0-logloss:0.45938
[6]	validation_0-logloss:0.43099
[7]	validation_0-logloss:0.40474
[8]	validation_0-logloss:0.38042
[9]	validation_0-logloss:0.35786
[10]	validation_0-logloss:0.33690
[11]	validation_0-logloss:0.31737
[12]	validation_0-logloss:0.29917
[13]	validation_0-logloss:0.28218
[14]	validation_0-logloss:0.26629
[15]	validation_0-logloss:0.25142
[16]	validation_0-logloss:0.23749
[17]	validation_0-logloss:0.22443
[18]	validation_0-logloss:0.21217
[19]	validation_0-logloss:0.20067
[20]	validation_0-logloss:0.18984
[21]	validation_0-logloss:0.17967
[22]	validation_0-logloss:0.17008
[23]	validation_0-logloss:0.16105
[24]	validation_0-logloss:0.15254
[25]	validation_0-logloss:0.14451
[26]	validation_0-logloss:0.13694
[27]	validation_0-logloss:0.12979
[28]	validation_0-logloss:0.12304
[29]	validation_0-loglos

In [7]:
+9

9